# Gng2 quantification

In [ ]:
# Import packages 
import sys
import numpy as np
import pandas as pd
import seaborn as sns
from natsort import natsorted
from scipy.io import loadmat, savemat
from skimage.filters import threshold_otsu
from skimage.color import label2rgb
from tqdm.notebook import tqdm
from skimage.feature import peak_local_max
from skimage.morphology import binary_dilation, disk
from skimage.util import regular_seeds, img_as_ubyte
from skimage.segmentation import watershed
from skimage.measure import regionprops_table

import anndata as ad
import scanpy as sc
# Customized packages 
from starmap.sequencing import *

In [ ]:
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42

## Path

In [ ]:
# IO path 
base_path = 'Z:/Data/Analyzed/2022-09-05-Hu-Tissue/'
# input_path = os.path.join(base_path, 'other-datasets/Gng2_protein_quantification')
input_path = os.path.join(base_path, 'other-datasets/Gng2_ISH_quantification')
fig_path = os.path.join(base_path, 'figures')

## Input

In [ ]:
# load protein image 
# protein = load_nissl_image(input_path, fname=f"Gng2_uint8.tif")
protein = load_nissl_image(input_path, fname=f"Gng2_ISH_quant_ref.tif")
protein.shape

In [ ]:
# # load segmentation image (protein)
# seg = load_nissl_image(input_path, fname=f"ref-labels.tif")
# # seg[seg == 1] = 0
# seg = seg[:2810, :2161]

# load segmentation image (ish)
seg = load_nissl_image(input_path, fname=f"Gng2_ISH_seg_ref-labels.tif")


seg.shape

In [ ]:
%%time

labels = []
protein_intensity = []
areas = []
# protein_pixel = []
# rna = []

# Iterate through cells
print('Iterate cells...')
for i, region in enumerate(tqdm(regionprops(seg))):
      # print(region.label)
    labels.append(region.label)
    areas.append(region.area)
    
    # protein intensity
    curr_intensity = np.sum(protein[region.coords[:, 0], region.coords[:, 1]])
    protein_intensity.append(round(curr_intensity, 2))
    
#     # protein pixel
#     curr_pixel = np.sum(protein_mask[region.coords[:, 0], region.coords[:, 1]])
#     protein_pixel.append(round(curr_pixel, 2))
        
#     # rna
#     curr_pixel = np.sum(dots_img[region.coords[:, 0], region.coords[:, 1]])
#     rna.append(round(curr_pixel, 2))

sns.histplot(areas)

In [ ]:
# # protein quantification df
# quantification_df = pd.DataFrame({"region_label": labels, "area": areas,  "protein_intensity": protein_intensity})
# quantification_df['protein_intensity_norm'] = quantification_df['protein_intensity'] / quantification_df['area']

# # ISH quantification df
quantification_df = pd.DataFrame({"region_label": labels, "area": areas,  "ish_intensity": protein_intensity})
quantification_df['ish_intensity_norm'] = quantification_df['ish_intensity'] / quantification_df['area']
quantification_df

In [ ]:
# map region
label_dict = {1: 'Isocortex', 2: 'HIP', 3: 'TH', 4: 'HY', 
             5: 'Fiber tracts', 6: 'CNU', 7: 'CTXsp', 8: 'OLF'}

In [ ]:
quantification_df['region_name'] = quantification_df['region_label']
quantification_df['region_name'] = quantification_df['region_name'].map(label_dict)
quantification_df

In [ ]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
df_scaled = std_scaler.fit_transform(quantification_df.ish_intensity_norm.to_numpy().reshape([8, 1]))
# df_scaled = pd.DataFrame(df_scaled, columns=df_te.index, index=df_te.columns)
quantification_df['ish_intensity_scaled'] = df_scaled
quantification_df

In [ ]:
seg_scaled = seg.copy()
seg_scaled = seg_scaled.astype('float')

for i, current_label in enumerate(quantification_df['region_label']):
    print(current_label)
    seg_scaled[np.where(seg == current_label)] = quantification_df.loc[i, 'ish_intensity_scaled']

In [ ]:
vmin = quantification_df['ish_intensity_norm'].min()
vmin

In [ ]:
fig, ax = plt.subplots(figsize=(28, 21))
plt.imshow(seg_scaled, cmap='bwr',)
plt.axis('off')
plt.colorbar()
# plt.savefig(os.path.join(fig_path, 'heatmap_Gng2_ish_quantification_manual.pdf'))

In [ ]:
from datetime import datetime
out_path = 'Z:/Data/Analyzed/2022-09-05-Hu-Tissue/output'
date = datetime.today().strftime('%Y-%m-%d')
quantification_df.to_csv(os.path.join(out_path, f"{date}-Gng2-ish-quantification.csv"))